# Information about Notebook
- Referenced https://github.com/fizyr/keras-retinanet.git
- Google Images v6 images used in conjunction with the dataset provided

# Environment Set Up

In [ ]:
!git clone https://github.com/fizyr/keras-retinanet.git

%cd keras-retinanet/
!pip install .

!python setup.py build_ext --inplace

In [ ]:
import os
import shutil
import zipfile
import urllib
import xml.etree.ElementTree as ET
import numpy as np
import csv
import math
import pandas
import json
import cv2
import tqdm

In [ ]:
annotations = []
valannotations = []
classes = ['tops','trousers','outerwear','dresses','skirts']

for i, line in enumerate(classes):
    print('{},{}'.format(line,i))

#FOR ANNOTATIONS
with open('/kaggle/input/tildata-combined1/train_combined_cleaned.json', mode='r') as json_file:
    data = json.load(json_file)
    for row in data:
        left,top,width,height = row['bbox']
        item = ['/kaggle/input/tildata-combined1/train_combined/train_combined/'+str(row['image_id'])+'.jpg'] + [left, top, left+width, top+height] + [classes[row['category_id'] - 1]]
        annotations.append(item)

#FOR VALIDATION ANNOTATIONS
with open('/kaggle/input/retinanetcsvfiles/validation.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        item = ['/kaggle/input/til2020/'+row['filename']] + [math.floor(float(row['xmin'])),math.floor(float(row['ymin'])),math.floor(float(row['xmax'])),math.floor(float(row['ymax']))] + [row['class']]
        valannotations.append(item)

In [ ]:
ANNOTATIONS_FILE = '/kaggle/working/annotations.csv'
VALANNOTATIONS_FILE = '/kaggle/working/valannotations.csv'
CLASSES_FILE = '/kaggle/working/classes.csv'

with open(ANNOTATIONS_FILE, 'w') as f:
  writer = csv.writer(f)
  writer.writerows(annotations)

with open(CLASSES_FILE, 'w') as f:
  writer = csv.writer(f)
  for i, line in enumerate(classes):
    f.write('{},{}\n'.format(line,i))
  
with open(VALANNOTATIONS_FILE, 'w') as f:
  writer = csv.writer(f)
  writer.writerows(valannotations)

# Training

In [ ]:
PRETRAINED_MODEL = './snapshots/resnet101_pretrained_model.h5'

DOWNLOAD INITIAL PRETRAINED MODEL FROM FIZYR
URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet101_oid_v1.0.0.h5'

urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

In [ ]:
TRAIN = "./keras_retinanet/bin/train.py"

#PRETRAINED_MODEL = './snapshots/resnet101_pretrained_model.h5'
SNAPSHOT1 = "../../input/retinanet-trained-models/resnet101combined10epochs_snapshot.h5"

VALANNOTATIONS = "../valannotations.csv"
ANNOTATIONS = "../annotations.csv" 
CLASSES = "../classes.csv"

In [ ]:
!python {TRAIN}  --snapshot {SNAPSHOT1} --backbone {'resnet101'} --image-min-side 400 --image-max-side 700 --freeze-backbone --random-transform --batch-size 8 --steps 500 --epochs 5 csv {ANNOTATIONS} {CLASSES}

# Evaluation

Convert training model to inference model

In [ ]:
#change the file paths of the snapshot and where to save the inference model to
model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[1])
print(model_path)

CONVERT = './keras_retinanet/bin/convert_model.py'
SNAPSHOT2 = model_path
INFERENCEMODEL = './snapshots/output.h5'

In [ ]:
!python './keras_retinanet/bin/convert_model.py' {model_path} './snapshots/output.h5'

Evauation using built-in mAP

In [ ]:
EVALUATE = './keras_retinanet/bin/evaluate.py'

INFERENCEMODEL = './snapshots/output.h5'

VALANNOTATIONS = "../valannotations.csv"
ANNOTATIONS = "../annotations.csv" 
CLASSES = "../classes.csv"

In [ ]:
!python {EVALUATE} --iou-threshold 0.5 --max-detections 100 csv {VALANNOTATIONS} {CLASSES} {INFERENCEMODEL}

Evaluation using COCOapi

In [ ]:
os.getcwd()

In [ ]:
from keras_retinanet.models import load_model
model = load_model('snapshots/output.h5', backbone_name='resnet101')

In [ ]:
%cd '/kaggle/working/keras-retinanet'

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

In [ ]:
detections = []
score_threshold = 0.

for root, dirs, files in os.walk('../../input/til2020/val/val'):
    i = 0
    for filename in files:
        #if i == 1:
          #break
        img = '../../input/til2020/val/val/'+ filename
        image = read_image_bgr(img)
          
        # preprocess image for network
        image = preprocess_image(image)
        image, scale = resize_image(image)

        # process image
        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))

        # correct for image scale
        boxes /= scale
        i+=1
        x = 0
        for box, score, label in zip(boxes[0], scores[0], labels[0]):
          #print(list(box))
          if x == 100:
            break
          elif float(score) < score_threshold:
            break

          x +=1
          cat_id = int(label)
          conf = float(score)
          bbox = list(box)
          left = round(float(bbox[0]),1)
          top = round(float(bbox[1]),1)
          width = round(float(bbox[2] - bbox[0]),1)
          height = round(float(bbox[3] - bbox[1]),1)
          img_id = img.split("/")[-1]
          img_id = int(img_id.split(".")[0])

          detections.append( {'image_id':img_id, 'category_id':cat_id, 'bbox':[left, top, width, height], 'score':conf} )
    print("done")

In [ ]:
print(detections[0])

In [ ]:
detections2 = detections
for detection in detections2:
    if detection['category_id'] == 0:
        detection['category_id'] = 1
    elif detection['category_id'] == 1:
        detection['category_id'] = 2
    elif detection['category_id'] == 2:
        detection['category_id'] = 3
    elif detection['category_id'] == 3:
        detection['category_id'] = 4
    elif detection['category_id'] == 4:
        detection['category_id'] = 5

print(detections2[0])

In [ ]:
import json
with open('./retina_combined15epochs.json', 'w') as f:
  json.dump(detections2, f)

In [ ]:
! python -m pip install numpy==1.17.1
! pip install git+https://github.com/jinmingteo/cocoapi.git#subdirectory=PythonAPI

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [ ]:
coco_gt = COCO('/kaggle/input/til2020/val.json')
coco_dt = coco_gt.loadRes('/kaggle/working/keras-retinanet/retina_combined15epochs.json')
cocoEval = COCOeval(cocoGt=coco_gt, cocoDt=coco_dt, iouType='bbox')
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()